In [1]:
import os
import h5py
import json
import mmap
import pickle
import numpy as np
from tqdm.auto import tqdm
from numba.typed import Dict, List
from numba import types
from transformers import (
    BertTokenizer, AutoModelForMaskedLM
)

/opt/conda/envs/ptca/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("/datacosmos/local/User/baoht/onesparse2/hybridsearch/bm25/index/marco/index_dist.json") as f:
    bm25_en_dist = json.load(f)

In [12]:
bm25_en_dist = dict(sorted(bm25_en_dist.items(), key=lambda item: int(item[0])))

In [15]:
dict_temp = {}
for k, v in bm25_en_dist.items():
    dict_temp[int(k)] = v
dict_temp = dict(sorted(dict_temp.items(), key=lambda item: item[0]))

In [ ]:
total = 0
filename = "/datacosmos/local/User/baoht/onesparse2/marcov2/data/collection_train"
file_lists = [os.path.join(filename, f'part-{i}') for i in range(20)]

def count_lines(filename):
    cnt = 0 
    with open(filename, 'rb') as f:
        for line in f:
            cnt += 1
    return cnt
            
for file in file_lists:
    lens = count_lines(file)
    total += lens
    print("File {} has {} lines".format(file, lens))
print(total)

In [6]:
import torch

weight = torch.ones(105879)
weight[:173]  = 0
with open("models/sparse_vocab_weight.tensor", "wb") as f:
    torch.save(weight, f)

In [36]:
import lmdb
import pickle

base_dir = "/datacosmos/User/baoht/onesparse2/marcov2/"
train_gt_path = "data/qrels_test.tsv"
gt_dict = {}
with open(base_dir + train_gt_path, 'r') as f:
    for l in f:
        try:
            l = l.strip().split('\t')
            qid = int(l[0])
            if qid in gt_dict:
                pass
            else:
                gt_dict[qid] = []
            gt_dict[qid].append(int(l[2]))
        except:
            raise IOError('\"%s\" is not valid format' % l)
            

In [13]:
train_questions = {}
with open(base_dir + "data/queries_train.tsv", "r", encoding="utf-8") as ifile:
    for line in tqdm(ifile):
        line = line.strip().split('\t')
        id, text, language = line
        train_questions[int(id)] = text

9206475it [00:10, 858297.22it/s]


In [18]:
passages_train_path = base_dir + "data/training_data/test_lmdb_new"
doc_pool_env = lmdb.open(passages_train_path, subdir=os.path.isdir(passages_train_path), readonly=True, lock=False, readahead=False, meminit=False)
passages_txn = doc_pool_env.begin(write=False)

In [38]:
gt_dict[1]
text = pickle.loads(passages_txn.get(("185462934").encode()))

In [39]:
text

['',
 'uin-\ue000app\ue001 的条件编译（\ue000APP-PLUS\ue001 、H5、MP-WEIXIN ...',
 'uin-app 的条件编译（APP-PLUS 、H5、MP-WEIXIN ）_leoliyou的博客-CSDN博客#N#uin-app 的条件编译（APP-PLUS 、H5、MP-WEIXIN ）#N#leoliyou 于 2020-07-29 15:48:02 发布 6203 收藏 22#N#分类专栏： Uin-app#N#版权#N#Uin-app 专栏收录该内容#N#1 篇文章 0 订阅#N#订阅专栏#N#一、条件编译#N#条件编译是用特殊的注释作为标记，在编译时根据这些特殊的注释，将注释里面的代码编译到不同平台。#N#写法：以 #ifdef 或 #ifndef 加 %PLATFORM% 开头，以 #endif 结尾。#N#1 #ifdef：if defined 仅在某平台存在#N#2 #ifndef：if not defined 除了某平台均存在#N#3 %PLATFORM%：平台名称#N#条件编译写法 说明#N##ifdef APP-PLUS#N#需条件编译的代码 仅出现在 App 平台下的代码#N##endif#N##ifndef H5#N#需条件编译的代码 除了 H5 平台，其它平台均存在的代码#N##endif#N##ifdef H5 || MP-WEIXIN#N#需条件编译的代码 在 H5 平台或微信小程序平台存在的代码（这里只有||，不可能出现&&，因为没有交集）#N##endif#N#值 平台#N#APP-PLUS App#N#APP-PLUS-NVUE App nvue#N#H5 H5#N#MP-WEIXIN 微信小程序#N#MP-ALIPAY 支付宝小程序#N#MP-BAIDU 百度小程序#N#MP-TOUTIAO 字节跳动小程序#N#MP-QQ QQ小程序#N#MP-360 360小程序#N#MP 微信小程序/支付宝小程序/百度小程序/字节跳动小程序/QQ小程序/360小程序#N#quickapp-webview 快应用通用(包含联盟、华为)#N#quickapp-webview-union 快应用联盟#N#quickapp-webview-huawei 快应用华为#N#以上在兼容H5、ap

In [1]:
from transformers import (
    BertTokenizerFast
)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-uncased")
tokenizer.encode(query)

/opt/conda/envs/ptca/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


NameError: name 'query' is not defined

In [41]:
with open("splade_index/warmup/shard_0/doc_ids_splade_index.bin.pkl", "rb") as f:
    doc_ids = pickle.load(f)

In [42]:
doc_ids[:10]

['159555704',
 '196055807',
 '196055816',
 '206797042',
 '196055823',
 '120904217',
 '129494639',
 '178877602',
 '155262280',
 '144529003']

In [3]:
with open("splade_index/warmup/shard_0/doc_ids_splade_index.bin.pkl", 'rb') as f:
    doc_ids = pickle.load(f)

In [6]:
import lmdb
import pickle

base_dir = "/datacosmos/User/baoht/onesparse2/marcov2/"
train_gt_path = "data/qrels_test.tsv"
gt_dict = {}
with open(base_dir + train_gt_path, 'r') as f:
    for l in f:
        try:
            l = l.strip().split('\t')
            qid = int(l[0])
            if qid in gt_dict:
                pass
            else:
                gt_dict[qid] = []
            gt_dict[qid].append(int(l[2]))
        except:
            raise IOError('\"%s\" is not valid format' % l)
            
gt_set = set()
for k, v in gt_dict.items():
    for pid in v:
        gt_set.add(int(pid))
print(len(gt_set))

9373


In [8]:
cnt = 0
for pid in doc_ids:
    if int(pid) in gt_set:
        cnt += 1
print(cnt)

1839


In [9]:
gt_dict[1]

[185462934]

In [5]:
qe = {"text_id": 1, "text": [10282, 35821, 10608, 108, 10481, 11526, 116, 39475, 59906, 19441, 39520, 11985, 15386, 14020, 51846, 69930, 23588, 85698, 12866, 24296, 21105, 26157, 78061, 17376, 53368, 82787, 22850, 13145, 49559, 56721, 58787, 20460], "value": [1.6295467615127563, 1.5897133350372314, 1.5630666017532349, 1.477690577507019, 1.35756516456604, 1.3565596342086792, 1.0506030321121216, 0.7076893448829651, 0.562402069568634, 0.5415972471237183, 0.5232481360435486, 0.48670902848243713, 0.4450363218784332, 0.4196864068508148, 0.409363716840744, 0.3910386264324188, 0.347847044467926, 0.31204116344451904, 0.31132611632347107, 0.3034266233444214, 0.24018500745296478, 0.20102573931217194, 0.18167029321193695, 0.151916041970253, 0.14095008373260498, 0.13414156436920166, 0.09761575609445572, 0.08244366943836212, 0.08244366943836212, 0.05046287178993225, 0.012615403160452843, 0.007782140746712685]}
# {"text_id": 2, "text": [110, 50017, 122, 1596, 94405, 20187, 30645, 82787, 39520, 11937, 29562, 39475, 18612, 31725, 13409, 46983, 85698, 13121, 95009, 26157, 41256, 1651, 20098, 92229, 20910, 18465, 51846, 51448, 84099, 10399, 10889, 0], "value": [2.068641185760498, 1.4918500185012817, 1.4283655881881714, 1.1530905961990356, 1.0894558429718018, 0.8161250352859497, 0.6902132034301758, 0.6064908504486084, 0.6054252982139587, 0.5278670787811279, 0.5104349255561829, 0.48068851232528687, 0.472198486328125, 0.4374985992908478, 0.3757305145263672, 0.35677260160446167, 0.3325583338737488, 0.23247487843036652, 0.2018241435289383, 0.175953671336174, 0.1477125883102417, 0.1212492436170578, 0.10555581003427505, 0.10555581003427505, 0.04767346754670143, 0.034552380442619324, 0.034552380442619324, 0.0317181795835495, 0.025073640048503876, 0.023167060688138008, 0.0038986406289041042, 0.0]}


In [6]:
from transformers import (
    BertTokenizerFast
)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-uncased")

/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
decoded_text = tokenizer.decode(qe["text"], skip_special_tokens=True)
print(decoded_text)

##de app plus #f if + xml sql linux cpu pdf microsoft software gsm tracking twitter hilbert windows bmw facebook itunes bundestag administrative playboy iupac legislative standard fenster rpg layout boeing


In [2]:
model_name =  "google-bert/bert-base-multilingual-uncased"
mlm_model = AutoModelForMaskedLM.from_pretrained(model_name)

Using the `SDPA` attention implementation on multi-gpu setup with ROCM may lead to performance issues due to the FA backend. Disabling it to use alternative backends.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will b

In [5]:
mlm_model.cls.state_dict()['predictions.bias']

tensor([-0.0897, -0.0904, -0.0862,  ..., -0.1417, -0.1358, -0.1432])

In [7]:
from model_zoo import BiEncoder
from auxiliary import ModelConfig
model_config = ModelConfig(
    model_name_or_path="models/init_SimANS_ckpt_36k",
    vocab_size=105879
)
model = BiEncoder(model_config)

In [12]:
print(model.k_encoder.mlm_head.state_dict()['predictions.bias'])
print(model.q_encoder.mlm_head.state_dict()['predictions.bias'])

tensor([-0.0897, -0.0904, -0.0862,  ..., -0.1417, -0.1358, -0.1432])
tensor([-0.0897, -0.0904, -0.0862,  ..., -0.1417, -0.1358, -0.1432])
